In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
site_df = pd.read_csv('data/abcd_y_lt.csv').iloc[:,[0,2,7]]
testdata = pd.read_csv('data/test.csv')
testdata = testdata[testdata.columns[~testdata.columns.str.startswith("cbcl")]]
traindata = pd.read_csv('data/train.csv')
traindata = traindata[traindata.columns[~traindata.columns.str.startswith("cbcl")]]
lostcolumns= ["interview_date","site","src_subject_id","y_t", "y_{t+1}", "eventname","race_ethnicity","urban"]
site_df['interview_date'] = pd.to_datetime(site_df['interview_date'], errors = 'coerce')
traindata['interview_date'] = pd.to_datetime(traindata['interview_date'], errors = 'coerce')
testdata['interview_date'] = pd.to_datetime(testdata['interview_date'], errors = 'coerce')
site_df['site_id_l'] = site_df['site_id_l'].str[4:].astype(int)
subject_date_site_mapping = (
    site_df.groupby('src_subject_id')
    .apply(lambda x: dict(zip(x['interview_date'], x['site_id_l'])))
    .to_dict()
)
def get_site(row):
    subject_id = row['src_subject_id']
    interview_date = row['interview_date']
    return subject_date_site_mapping.get(subject_id, {}).get(interview_date, None)

df_urban = pd.read_csv("data/led_l_urban.csv")
traindata['site'] = traindata.apply(get_site, axis=1)
traindata.insert(1, 'site', traindata.pop('site'))
traindata = traindata.merge(
    df_urban[['src_subject_id', 'reshist_addr1_urban_area']],
    on='src_subject_id',
    how='left'  
)
traindata.insert(1, 'urban', traindata.pop('reshist_addr1_urban_area'))


testdata['site'] = traindata.apply(get_site, axis=1)
testdata.insert(1, 'site', testdata.pop('site'))  
testdata = testdata.merge(
    df_urban[['src_subject_id', 'reshist_addr1_urban_area']],
    on='src_subject_id',
    how='left' 
)
testdata.insert(1, 'urban', testdata.pop('reshist_addr1_urban_area'))  

testdata = testdata.apply(pd.to_numeric,errors='coerce')

traindata = traindata.apply(pd.to_numeric,errors='coerce')


top_vars = [x for x in testdata.columns if x not in lostcolumns]


testdata['white'] = (testdata['race_ethnicity'] == 1).astype(int)
testdata['black'] = (testdata['race_ethnicity'] == 2).astype(int)
testdata['hispano'] = (testdata['race_ethnicity'] == 3).astype(int)
testdata['asian'] = (testdata['race_ethnicity'] == 4).astype(int)
testdata['others'] = (testdata['race_ethnicity'] == 5).astype(int)


traindata['white'] = (traindata['race_ethnicity'] == 1).astype(int)
traindata['black'] = (traindata['race_ethnicity'] == 2).astype(int)
traindata['hispano'] = (traindata['race_ethnicity'] == 3).astype(int)
traindata['asian'] = (traindata['race_ethnicity'] == 4).astype(int)
traindata['others'] = (traindata['race_ethnicity'] == 5).astype(int)

data_train, hatdata = train_test_split(traindata, test_size=0.2, random_state=42)

data_test =testdata

X_train_df = data_train.drop(columns=lostcolumns)
X_test_df = data_test.drop(columns=lostcolumns)
feature_names = X_train_df.columns

/var/folders/47/91sqyf453hs814g86jzgnwdh0000gn/T/ipykernel_72986/3692189692.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: dict(zip(x['interview_date'], x['site_id_l'])))


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
import numpy as np
class MultiLayerPerceptron(nn.Module):
    def __init__(self, input_dim, num_classes, hidden_dim=64, dropout_rate=0.3):
        super(MultiLayerPerceptron, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        return self.model(x)

class ModelTrainer:
    def __init__(self, input_dim, num_classes=4, learning_rate=0.01, num_epochs=6000):
        self.input_dim = input_dim
        self.num_classes = num_classes
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.model = None
        self.best_model_params = None

    def _prepare_data(self, X, y):
        X_tensor = torch.tensor(X, dtype=torch.float32)
        y_tensor = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)
        return X_tensor, y_tensor

    def evaluate(self, X_test, y_test):
        self.model.eval()
        with torch.no_grad():
            y_pred = self.model(torch.tensor(X_test, dtype=torch.float32))
            y_prob = F.softmax(y_pred, dim=1).numpy()
        auc_value = roc_auc_score((y_test == 3).astype(int), y_prob[:, 3])
        return auc_value, y_prob[:, 3]

class TraditionalMethod(ModelTrainer):
    def __init__(self, input_dim, num_classes=4, learning_rate=0.01, num_epochs=6000):
        super().__init__(input_dim, num_classes, learning_rate, num_epochs)
        self.model = MultiLayerPerceptron(input_dim=input_dim, num_classes=num_classes)
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

    def train(self, X_train, y_train):
        X_train_tensor, y_train_tensor = self._prepare_data(X_train, y_train)
        for epoch in range(self.num_epochs):
            self.optimizer.zero_grad()
            logits = self.model(X_train_tensor)
            loss = F.cross_entropy(logits, y_train_tensor.view(-1).long(), reduction='mean')
            loss.backward()
            self.optimizer.step()
        self.best_model_params = {name: param.clone() for name, param in self.model.named_parameters()}
        self.model.load_state_dict(self.best_model_params)
        return self

class RegularizationMethod(ModelTrainer):
    def __init__(self, input_dim, num_classes=4, learning_rate=0.01, num_epochs=6000):
        super().__init__(input_dim, num_classes, learning_rate, num_epochs)
        self.model = MultiLayerPerceptron(input_dim=input_dim, num_classes=num_classes)
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)
        self.lambdas = [0.000001,0.000005,0.00001,0.00005,0.0001, 0.001, 0.005,0.01,0.05]
        self.best_lambda = None

    def train(self, X_train, y_train, hatdata, top_vars):
        X_train_tensor, y_train_tensor = self._prepare_data(X_train, y_train)
        best_auc = -float("inf")
        for lambda_val in self.lambdas:
            model = MultiLayerPerceptron(input_dim=self.input_dim, num_classes=self.num_classes)
            optimizer = optim.Adam(model.parameters(), lr=self.learning_rate)
            for epoch in range(self.num_epochs):
                optimizer.zero_grad()
                logits = model(X_train_tensor)
                loss = F.cross_entropy(logits, y_train_tensor.view(-1).long(), reduction='mean')
                l2_reg = lambda_val * sum(p.pow(2).sum() for p in model.parameters())
                loss += l2_reg
                loss.backward()
                optimizer.step()
            model.eval()
            with torch.no_grad():
                X_val = hatdata[top_vars].values
                y_val = hatdata["y_{t+1}"].values
                y_pred = model(torch.tensor(X_val, dtype=torch.float32))
                y_prob = F.softmax(y_pred, dim=1).numpy()
                val_auc = roc_auc_score((y_val == 3).astype(int), y_prob[:, 3])
                if val_auc > best_auc:
                    best_auc = val_auc
                    self.best_lambda = lambda_val
                    self.best_model_params = {name: param.clone() for name, param in model.named_parameters()}
                print(best_auc)
        self.model.load_state_dict(self.best_model_params)
        return self

class DROMethod(ModelTrainer):
    def __init__(self, input_dim, num_classes=4, learning_rate=0.01, num_epochs=6000):
        super().__init__(input_dim, num_classes, learning_rate, num_epochs)
        self.model = MultiLayerPerceptron(input_dim=input_dim, num_classes=num_classes)
        self.kappa_iter = [1e-7,3e-7,1e-6,2e-6, 3e-6,5e-6,7e-6,1e-5,5e-5,1e-4,5e-4,1e-3]
        self.kappacoef = [1,1.2]
        self.best_kappa = None
        self.was = [18]
        self.best_kappacoef = None
        self.best_lambda = None
        self.scaler = StandardScaler()



    def train_single_run(self, X_train, y_train, hatdata, top_vars):
        X_train_values = X_train
        X_train_tensor = torch.tensor(X_train_values, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
        best_auc = -float("inf")
        for kcoefiter in self.kappacoef:
            for kappa in self.kappa_iter:
                for wasserstein in self.was:
                    model = MultiLayerPerceptron(input_dim=self.input_dim, num_classes=self.num_classes)
                    optimizer = optim.Adam(model.parameters(), lr=self.learning_rate)
                    lambda_param_raw = torch.nn.Parameter(torch.tensor(1.0))
                    optimizer_lambda = optim.Adam([lambda_param_raw], lr=self.learning_rate)
                    for epoch in range(self.num_epochs):
                        optimizer.zero_grad()
                        optimizer_lambda.zero_grad()
                        logits = model(X_train_tensor)
                        loss = F.cross_entropy(logits, y_train_tensor.view(-1).long())
                        l2_norm = torch.sqrt(sum(p.pow(2).sum() for p in model.parameters()))
                        lambda_param = torch.exp(lambda_param_raw)
                        y_onehot = torch.eye(logits.size(1))[y_train_tensor.view(-1).long()].to(X_train_tensor.device)
                        true_logits = torch.sum(y_onehot * logits, dim=1)
                        max_other_logits = torch.max((1 - y_onehot) * logits, dim=1)[0]
                        margins = true_logits - max_other_logits - lambda_param * kappa
                        label_uncertainty_term = torch.relu(margins).mean()
                        loss += wasserstein * lambda_param
                        loss += kcoefiter * label_uncertainty_term
                        penalty = torch.relu(l2_norm - lambda_param).pow(2)
                        loss += 500 * penalty
                        loss.backward()
                        optimizer.step()
                        optimizer_lambda.step()
                        with torch.no_grad():
                            lambda_param_raw.data = torch.log(torch.max(torch.exp(lambda_param_raw), l2_norm + 1e-6))
                            lambda_param_raw.data.clamp_(min=-10)
                    model.eval()
                    with torch.no_grad():
                        X_val = hatdata[top_vars].values
                        y_val = hatdata["y_{t+1}"].values
                        y_pred = model(torch.tensor(X_val, dtype=torch.float32))
                        y_prob = F.softmax(y_pred, dim=1).numpy()
                        val_auc = roc_auc_score((y_val == 3).astype(int), y_prob[:, 3])
                        if val_auc > best_auc:
                            best_auc = val_auc
                            self.best_kappa = kappa
                            self.best_kappacoef = kcoefiter
                            self.best_lambda = lambda_param.data
                            self.best_model_params = {name: param.clone() for name, param in model.named_parameters()}
                            self.best_was = wasserstein
                        print(best_auc)
        self.model.load_state_dict(self.best_model_params)
        return self

In [ ]:
from sklearn.metrics import roc_curve, auc
X_train = X_train_df[top_vars].values
y_train = data_train["y_{t+1}"]
X_test = X_test_df[top_vars].values
y_test = data_test["y_{t+1}"].values

input_dim = X_train.shape[1]

print("TRA")
trad_model = TraditionalMethod(input_dim=input_dim)
trad_model.train(X_train, y_train)
trad_auc, trad_probs = trad_model.evaluate(X_test, y_test)
print("REG")
reg_model = RegularizationMethod(input_dim=input_dim)
reg_model.train(X_train, y_train, hatdata, top_vars)
reg_auc, reg_probs = reg_model.evaluate(X_test, y_test)
print(reg_auc)
print("Best lambda (Reg):", reg_model.best_lambda)


def train_dro_multiple_runs(n_runs, X_train, y_train, hatdata, top_vars, X_test, y_test):
    probs_list = []
    auc_list = []
    best_params = []
    models = []  

    for run in range(n_runs):
        print(run)
        model = DROMethod(input_dim=X_train.shape[1])
        model.train_single_run(X_train, y_train, hatdata, top_vars)
        auc_val, probs = model.evaluate(X_test, y_test)
        print(auc_val)

        probs_list.append(probs)
        auc_list.append(auc_val)
        models.append(model)

        print({
            'kappa': model.best_kappa,
            'kappacoef': model.best_kappacoef,
            'lambda': model.best_lambda.item(),
            'was': model.best_was
        })

        best_params.append({
            'kappa': model.best_kappa,
            'kappacoef': model.best_kappacoef,
            'lambda': model.best_lambda.item(),
            'was': model.best_was
        })

    avg_probs = np.mean(np.stack(probs_list), axis=0)
    fpr, tpr, _ = roc_curve((y_test == 3).astype(int), avg_probs)
    avg_auc = auc(fpr, tpr)

    return models, avg_auc, avg_probs, best_params




dro_models, dro_auc, dro_probs, dro_best_params = train_dro_multiple_runs(
    n_runs=2,
    X_train=X_train,
    y_train=y_train,
    hatdata=hatdata,
    top_vars=top_vars,
    X_test=X_test,
    y_test=y_test
)


state_dicts = [m.best_model_params for m in dro_models]


TRA
REG
0.7925418847768705
0.8001657172022137
0.8005434292700826
0.8007557732813466
0.8007557732813466
0.8007557732813466
0.8007557732813466
0.8007557732813466
0.8007557732813466
0.8007557732813466
0.8007557732813466
0.8007557732813466
0.8007557732813466
0.7941377638718242
Best lambda (Reg): 5e-05
0
0.5990703839459041
0.7926238546437001


In [ ]:
import matplotlib.pyplot as plt
y_true = (y_test == 3).astype(int)

fpr_dro, tpr_dro, _ = roc_curve(y_true, dro_probs)
auc_dro = auc(fpr_dro, tpr_dro)

trad_probs = trad_probs[:, 3]
fpr_trad, tpr_trad, _ = roc_curve(y_true, trad_probs)
auc_trad = auc(fpr_trad, tpr_trad)

reg_probs = reg_probs[:, 3]
fpr_reg, tpr_reg, _ = roc_curve(y_true, reg_probs)
auc_reg = auc(fpr_reg, tpr_reg)

plt.figure(figsize=(7, 7))
plt.plot(fpr_dro, tpr_dro, label=f"DRO (AUC = {auc_dro:.3f})", linewidth=2)
plt.plot(fpr_trad, tpr_trad, label=f"Traditional (AUC = {auc_trad:.3f})", linewidth=2)
plt.plot(fpr_reg, tpr_reg, label=f"Regularized (AUC = {auc_reg:.3f})", linewidth=2)

plt.plot([0, 1], [0, 1], 'k--', label='Random Guess')

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve Comparison (class 3)")
plt.legend(loc='lower right')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
trad_4_p = trad_probs
dro_4_p = dro_probs
reg_4_p = reg_probs
trad_4_p = trad_4_p.tolist()
reg_4_p = reg_4_p.tolist()
dro_4_p = dro_4_p.tolist()
indices_to_remove = testdata.index[testdata['y_t'] == 3].tolist()
testdata_4 = testdata
y_true_4 = y_true.tolist()

for idx in sorted(indices_to_remove, reverse=True):
    del trad_4_p[idx]
    del reg_4_p[idx]
    del dro_4_p[idx]
    del y_true_4[idx]

fpr_dro, tpr_dro, _ = roc_curve(y_true_4, dro_4_p)
auc_dro = auc(fpr_dro, tpr_dro)

fpr_trad, tpr_trad, _ = roc_curve(y_true_4, trad_4_p)
auc_trad = auc(fpr_trad, tpr_trad)

fpr_reg, tpr_reg, _ = roc_curve(y_true_4, reg_4_p)
auc_reg = auc(fpr_reg, tpr_reg)

plt.figure(figsize=(7, 7))
plt.plot(fpr_dro, tpr_dro, label=f"DRO (AUC = {auc_dro:.3f})", linewidth=2)
plt.plot(fpr_trad, tpr_trad, label=f"Traditional (AUC = {auc_trad:.3f})", linewidth=2)
plt.plot(fpr_reg, tpr_reg, label=f"Regularized (AUC = {auc_reg:.3f})", linewidth=2)

plt.plot([0, 1], [0, 1], 'k--', label='Random Guess')

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve Comparison (class 3)")
plt.legend(loc='lower right')
plt.grid(True)
plt.tight_layout()
plt.show()

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
import torch.optim as optim
from sklearn.preprocessing import StandardScaler


class MultiLayerPerceptron(nn.Module):
    def __init__(self, input_dim, num_classes, hidden_dim=64, dropout_rate=0.3):
        super(MultiLayerPerceptron, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, num_classes),
        )

    def forward(self, x):
        return self.model(x)


class ModelTrainer:
    def __init__(self, input_dim, num_classes=4, learning_rate=0.01, num_epochs=2000):
        self.input_dim = input_dim
        self.num_classes = num_classes
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.model = None
        self.best_model_params = None

    def _prepare_data(self, X, y):
        X_tensor = torch.tensor(X, dtype=torch.float32)
        y_tensor = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)
        return X_tensor, y_tensor

    def evaluate(self, X_test, y_test):
        self.model.eval()
        with torch.no_grad():
            y_pred = self.model(torch.tensor(X_test, dtype=torch.float32))
            y_prob = F.softmax(y_pred, dim=1).numpy()
        auc_value = roc_auc_score((y_test == 3).astype(int), y_prob[:, 3])
        return auc_value, y_prob[:, 3]


class TraditionalMethod(ModelTrainer):
    def __init__(self, input_dim, num_classes=4, learning_rate=0.01, num_epochs=4000):
        super().__init__(input_dim, num_classes, learning_rate, num_epochs)
        self.model = MultiLayerPerceptron(input_dim=input_dim, num_classes=num_classes)
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

    def train(self, X_train, y_train):
        X_train_tensor, y_train_tensor = self._prepare_data(X_train, y_train)
        for epoch in range(self.num_epochs):
            self.optimizer.zero_grad()
            logits = self.model(X_train_tensor)
            loss = F.cross_entropy(
                logits, y_train_tensor.view(-1).long(), reduction="mean"
            )
            loss.backward()
            self.optimizer.step()
        self.best_model_params = {
            name: param.clone() for name, param in self.model.named_parameters()
        }
        self.model.load_state_dict(self.best_model_params)
        return self


class RegularizationMethod(ModelTrainer):
    def __init__(self, input_dim, num_classes=4, learning_rate=0.01, num_epochs=4000):
        super().__init__(input_dim, num_classes, learning_rate, num_epochs)
        self.model = MultiLayerPerceptron(input_dim=input_dim, num_classes=num_classes)
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)
        self.lambdas = [
            0.0001,
            0.001,
            0.005,
            0.01,
            0.05,
            0.1,
            0.2,
            0.3,
            0.4,
        ]
        self.best_lambda = None

    def train(self, X_train, y_train, hatdata, top_vars):
        X_train_tensor, y_train_tensor = self._prepare_data(X_train, y_train)
        best_auc = -float("inf")
        for lambda_val in self.lambdas:
            model = MultiLayerPerceptron(
                input_dim=self.input_dim, num_classes=self.num_classes
            )
            optimizer = optim.Adam(model.parameters(), lr=self.learning_rate)
            for epoch in range(self.num_epochs):
                optimizer.zero_grad()
                logits = model(X_train_tensor)
                loss = F.cross_entropy(
                    logits, y_train_tensor.view(-1).long(), reduction="mean"
                )
                l2_reg = lambda_val * sum(p.pow(2).sum() for p in model.parameters())
                loss += l2_reg
                loss.backward()
                optimizer.step()
            model.eval()
            with torch.no_grad():
                X_val = hatdata[top_vars].values
                y_val = hatdata["y_{t+1}"].values
                y_pred = model(torch.tensor(X_val, dtype=torch.float32))
                y_prob = F.softmax(y_pred, dim=1).numpy()
                val_auc = roc_auc_score((y_val == 3).astype(int), y_prob[:, 3])
                if val_auc > best_auc:
                    best_auc = val_auc
                    self.best_lambda = lambda_val
                    self.best_model_params = {
                        name: param.clone() for name, param in model.named_parameters()
                    }
                print(best_auc)
        self.model.load_state_dict(self.best_model_params)
        return self


class DROMethod(ModelTrainer):
    def __init__(self, input_dim, num_classes=4, learning_rate=0.01, num_epochs=6000):
        super().__init__(input_dim, num_classes, learning_rate, num_epochs)
        self.model = MultiLayerPerceptron(input_dim=input_dim, num_classes=num_classes)
        self.kappa_iter = [
            2e-6,
            3e-6,
            5e-6,
            7e-6,
            1e-5,
            5e-5,
            1e-4,
            5e-4,
            1e-3,
        ]
        self.kappacoef = [1, 1.2]
        self.best_kappa = None
        self.was = [18]
        self.best_kappacoef = None
        self.best_lambda = None
        self.scaler = StandardScaler()

    def train_single_run(self, X_train, y_train, hatdata, top_vars):
        X_train_values = X_train
        X_train_tensor = torch.tensor(X_train_values, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
        best_auc = -float("inf")
        for kcoefiter in self.kappacoef:
            for kappa in self.kappa_iter:
                for wasserstein in self.was:
                    model = MultiLayerPerceptron(
                        input_dim=self.input_dim, num_classes=self.num_classes
                    )
                    optimizer = optim.Adam(model.parameters(), lr=self.learning_rate)
                    lambda_param_raw = torch.nn.Parameter(torch.tensor(1.0))
                    optimizer_lambda = optim.Adam(
                        [lambda_param_raw], lr=self.learning_rate
                    )
                    for epoch in range(self.num_epochs):
                        optimizer.zero_grad()
                        optimizer_lambda.zero_grad()
                        logits = model(X_train_tensor)
                        loss = F.cross_entropy(logits, y_train_tensor.view(-1).long())
                        l2_norm = torch.sqrt(
                            sum(p.pow(2).sum() for p in model.parameters())
                        )
                        lambda_param = torch.exp(lambda_param_raw)
                        y_onehot = torch.eye(logits.size(1))[
                            y_train_tensor.view(-1).long()
                        ].to(X_train_tensor.device)
                        true_logits = torch.sum(y_onehot * logits, dim=1)
                        max_other_logits = torch.max((1 - y_onehot) * logits, dim=1)[0]
                        margins = true_logits - max_other_logits - lambda_param * kappa
                        label_uncertainty_term = torch.relu(margins).mean()
                        loss += wasserstein * lambda_param
                        loss += kcoefiter * label_uncertainty_term
                        penalty = torch.relu(l2_norm - lambda_param).pow(2)
                        loss += 500 * penalty
                        loss.backward()
                        optimizer.step()
                        optimizer_lambda.step()
                        with torch.no_grad():
                            lambda_param_raw.data = torch.log(
                                torch.max(torch.exp(lambda_param_raw), l2_norm + 1e-6)
                            )
                            lambda_param_raw.data.clamp_(min=-10)
                    model.eval()
                    with torch.no_grad():
                        X_val = hatdata[top_vars].values
                        y_val = hatdata["y_{t+1}"].values
                        y_pred = model(torch.tensor(X_val, dtype=torch.float32))
                        y_prob = F.softmax(y_pred, dim=1).numpy()
                        val_auc = roc_auc_score((y_val == 3).astype(int), y_prob[:, 3])
                        if val_auc > best_auc:
                            best_auc = val_auc
                            self.best_kappa = kappa
                            self.best_kappacoef = kcoefiter
                            self.best_lambda = lambda_param.data
                            self.best_model_params = {
                                name: param.clone()
                                for name, param in model.named_parameters()
                            }
                            self.best_was = wasserstein
                    print(best_auc)
        self.model.load_state_dict(self.best_model_params)
        return self


def train_and_eval_traditional(X_train, y_train, data, top_vars, filtered_list):
    """训练并评估传统方法"""
    input_dim = X_train.shape[1]
    model = TraditionalMethod(input_dim=input_dim)
    model.train(X_train, y_train)

    auc_list, pro_list = [], []
    auc_list_4, pro_list_4 = [], []

    for site in filtered_list:
        df_site = data[data["site"] == site]
        X_site = df_site[top_vars].values
        y_site = df_site["y_{t+1}"].values
        auc, proba = model.evaluate(X_site, y_site)
        auc_list.append(auc)
        pro_list.append(proba.reshape(-1, 1))

        df_site_4 = df_site[df_site["y_t"] != 3]
        X_site_4 = df_site_4[top_vars].values
        y_site_4 = df_site_4["y_{t+1}"].values
        auc_4, proba_4 = model.evaluate(X_site_4, y_site_4)
        auc_list_4.append(auc_4)
        pro_list_4.append(proba_4.reshape(-1, 1))

    return model, auc_list, pro_list, auc_list_4, pro_list_4


def train_and_eval_regularization(
    X_train, y_train, hatdata, top_vars, data, filtered_list
):
    """训练并评估正则化方法"""
    input_dim = X_train.shape[1]
    model = RegularizationMethod(input_dim=input_dim)
    model.train(X_train, y_train, hatdata, top_vars)

    auc_list, pro_list = [], []
    auc_list_4, pro_list_4 = [], []
    white_list, white_list_4 = [], []

    for site in filtered_list:
        df_site = data[data["site"] == site]
        X_site = df_site[top_vars].values
        y_site = df_site["y_{t+1}"].values
        auc, proba = model.evaluate(X_site, y_site)
        auc_list.append(auc)
        pro_list.append(proba.reshape(-1, 1))

        df_site_4 = df_site[df_site["y_t"] != 3]
        X_site_4 = df_site_4[top_vars].values
        y_site_4 = df_site_4["y_{t+1}"].values
        auc_4, proba_4 = model.evaluate(X_site_4, y_site_4)
        auc_list_4.append(auc_4)
        pro_list_4.append(proba_4.reshape(-1, 1))

        df_white = df_site[df_site["race_ethnicity"] != 1]
        X_white = df_white[top_vars].values
        y_white = df_white["y_{t+1}"].values
        auc_white, _ = model.evaluate(X_white, y_white)
        white_list.append(auc_white)

        df_white_4 = df_white[df_white["y_t"] != 3]
        X_white_4 = df_white_4[top_vars].values
        y_white_4 = df_white_4["y_{t+1}"].values
        auc_white_4, _ = model.evaluate(X_white_4, y_white_4)
        white_list_4.append(auc_white_4)

    return model, auc_list, pro_list, auc_list_4, pro_list_4, white_list, white_list_4


def train_and_eval_dro_multiple(
    n_runs, X_train, y_train, hatdata, top_vars, data, filtered_list
):
    """多次训练并评估DRO方法，返回平均结果"""
    input_dim = X_train.shape[1]
    auc_list = None
    pro_list = None
    auc_list_4 = None
    pro_list_4 = None
    white_list = None
    white_list_4 = None
    best_params = []
    models = []

    for run in range(n_runs):
        print(run)
        model = DROMethod(input_dim=input_dim)
        model.train_single_run(X_train, y_train, hatdata, top_vars)
        models.append(model)

        run_auc, run_pro, run_auc_4, run_pro_4, run_white, run_white_4 = (
            [],
            [],
            [],
            [],
            [],
            [],
        )
        for site in filtered_list:
            df_site = data[data["site"] == site]
            X_site = df_site[top_vars].values
            y_site = df_site["y_{t+1}"].values
            auc, proba = model.evaluate(X_site, y_site)
            run_auc.append(auc)
            run_pro.append(proba.reshape(-1, 1))

            df_site_4 = df_site[df_site["y_t"] != 3]
            X_site_4 = df_site_4[top_vars].values
            y_site_4 = df_site_4["y_{t+1}"].values
            auc_4, proba_4 = model.evaluate(X_site_4, y_site_4)
            run_auc_4.append(auc_4)
            run_pro_4.append(proba_4.reshape(-1, 1))

            df_white = df_site[df_site["race_ethnicity"] != 1]
            X_white = df_white[top_vars].values
            y_white = df_white["y_{t+1}"].values
            auc_white, _ = model.evaluate(X_white, y_white)
            run_white.append(auc_white)

            df_white_4 = df_white[df_white["y_t"] != 3]
            X_white_4 = df_white_4[top_vars].values
            y_white_4 = df_white_4["y_{t+1}"].values
            auc_white_4, _ = model.evaluate(X_white_4, y_white_4)
            run_white_4.append(auc_white_4)

        # 累加结果
        if auc_list is None:
            auc_list = run_auc
            pro_list = run_pro
            auc_list_4 = run_auc_4
            pro_list_4 = run_pro_4
            white_list = run_white
            white_list_4 = run_white_4
        else:
            auc_list = [a + b for a, b in zip(auc_list, run_auc)]
            pro_list = [a + b for a, b in zip(pro_list, run_pro)]
            auc_list_4 = [a + b for a, b in zip(auc_list_4, run_auc_4)]
            pro_list_4 = [a + b for a, b in zip(pro_list_4, run_pro_4)]
            white_list = [a + b for a, b in zip(white_list, run_white)]
            white_list_4 = [a + b for a, b in zip(white_list_4, run_white_4)]

        best_params.append(
            {
                "kappa": model.best_kappa,
                "kappacoef": model.best_kappacoef,
                "lambda": float(model.best_lambda)
                if hasattr(model.best_lambda, "item")
                else model.best_lambda,
                "was": model.best_was,
            }
        )

    # 求平均
    auc_list = [x / n_runs for x in auc_list]
    pro_list = [x / n_runs for x in pro_list]
    auc_list_4 = [x / n_runs for x in auc_list_4]
    pro_list_4 = [x / n_runs for x in pro_list_4]
    white_list = [x / n_runs for x in white_list]
    white_list_4 = [x / n_runs for x in white_list_4]

    return (
        models,
        auc_list,
        pro_list,
        auc_list_4,
        pro_list_4,
        white_list,
        white_list_4,
        best_params,
    )



import optuna
from sklearn.model_selection import train_test_split
import torch.optim as optim


def tune_traditional_with_optuna(X_train_all, y_train_all, hatdata, top_vars, n_trials=30):
    """使用 Optuna 调参传统方法并返回最优模型和参数"""

    def objective(trial):
        # 超参数空间
        hidden_dim = trial.suggest_int("hidden_dim", 32, 256, step=32)
        dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)
        learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)

        # 拆分为训练 + 验证
        X_train, X_val, y_train, y_val = train_test_split(
            X_train_all, y_train_all, test_size=0.2, stratify=y_train_all, random_state=42
        )

        # 创建模型
        model = MultiLayerPerceptron(
            input_dim=X_train.shape[1],
            num_classes=4,
            hidden_dim=hidden_dim,
            dropout_rate=dropout_rate
        )
        trainer = TraditionalMethod(
            input_dim=X_train.shape[1],
            learning_rate=learning_rate,
            num_epochs=300  # 较小的epoch用于调参
        )
        trainer.model = model
        trainer.optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        trainer.train(X_train, y_train)

        auc, _ = trainer.evaluate(X_val, y_val)
        return auc

    # Optuna 搜索
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)

    # 用最佳参数重新训练（全epoch）
    best_params = study.best_params
    print("Best AUC:", study.best_value)
    print("Best params:", best_params)

    final_model = TraditionalMethod(
        input_dim=X_train_all.shape[1],
        learning_rate=best_params["learning_rate"],
        num_epochs=4000
    )
    final_model.model = MultiLayerPerceptron(
        input_dim=X_train_all.shape[1],
        num_classes=4,
        hidden_dim=best_params["hidden_dim"],
        dropout_rate=best_params["dropout_rate"]
    )
    final_model.optimizer = optim.Adam(
        final_model.model.parameters(), lr=best_params["learning_rate"]
    )
    final_model.train(X_train_all, y_train_all)

    return final_model, best_params


In [11]:
final_model, best_params = tune_traditional_with_optuna(
    X_train_all=X_train_df,
    y_train_all=data_train["y_{t+1}"],
    hatdata=hatdata,
    top_vars=top_vars,
    n_trials=30  # 可以改成你想要的搜索轮数
)


[I 2025-04-23 20:28:31,553] A new study created in memory with name: no-name-be23f1b0-ecec-4cce-841e-0284942668a7
[W 2025-04-23 20:28:32,138] Trial 0 failed with parameters: {'hidden_dim': 128, 'dropout_rate': 0.48630785620850503, 'learning_rate': 0.00016356825485146068} because of the following error: ValueError("could not determine the shape of object type 'DataFrame'").
Traceback (most recent call last):
  File "/Users/gaozhiyuan/.pyenv/versions/3.12.1/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/var/folders/47/91sqyf453hs814g86jzgnwdh0000gn/T/ipykernel_65217/3764960907.py", line 419, in objective
    trainer.train(X_train, y_train)
  File "/var/folders/47/91sqyf453hs814g86jzgnwdh0000gn/T/ipykernel_65217/3764960907.py", line 56, in train
    X_train_tensor, y_train_tensor = self._prepare_data(X_train, y_train)
                                     ^^^^^^^^^^^^^^^^^^^^^^^^^

ValueError: could not determine the shape of object type 'DataFrame'